In [1]:
using CSV, DataFrames, Random, HTTP, Distributions

In [2]:
url = "https://raw.githubusercontent.com/ArieBeresteanu/Bounds.jl/main/Fortran/Chtdata.csv"

http_response = HTTP.get(url)
file = CSV.File(http_response.body,header=false)

cht = DataFrame(file)

rename!(cht, :Column1 => :yl, :Column2 => :yu, :Column3 => :edu)
first(cht,3)

,yl,yu,edu
,Float64,Float64,Int64
1,4.0,4.0969,28
2,4.5441,4.699,28
3,4.1761,4.301,34


In [3]:
s = describe(cht, :mean, cols=[:yl,:yu])
H0 = [s[1,2], s[2,2]]

2-element Vector{Float64}:
 4.434663145221974
 4.967426358164033

In [10]:
Nobs = size(cht)[1]

13290

In [13]:
rng = setBLP.default_options.rng

MersenneTwister(15217)

In [4]:
include("C:\\Users\\The Group Leader\\Notebooks\\Research\\Bounds.jl-1\\src\\setBLP.jl")

Main.setBLP

In [41]:
N = [100,200,500,1000,2000]

MC = 25000
Rej_H = zeros(length(N))
Rej_dH = zeros(length(N))

for j=1:length(N)
    n=N[j]
    r_H, r_dH = 0,0
    for i=1:MC
        indx = rand(rng, 1:Nobs,n);
        res = setBLP.EY(cht[indx,:yl],cht[indx,:yu],H0)
        r_H += (res.Htest.testStat > res.Htest.criticalVal)
        r_dH += (res.dHtest.testStat > res.dHtest.criticalVal)
    end
    Rej_H[j] = r_H/MC
    Rej_dH[j] = r_dH/MC
end

In [43]:
[N Rej_H Rej_dH]

5×3 Matrix{Float64}:
  100.0  0.04188  0.06012
  200.0  0.04644  0.05828
  500.0  0.05072  0.05656
 1000.0  0.05136  0.05544
 2000.0  0.05064  0.05436